In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('switchboard.tsv', sep='\t')
print(df) 
print(df.columns)

In [ ]:
len(df)

In [ ]:
df['file']

In [ ]:
df.drop(['speaker', 'turn', 'sent_num', 'sentence', 'comb_sentence', 'names', 'ms_names', 'comb_ann', 'first_name'], axis=1, inplace=True)

In [ ]:
df['file']

In [ ]:
rows1 = df['file'].str.contains("sw[23]")
rows2= df['file'].str.contains("sw4[5-9]")
rows3 = df['file'].str.contains("sw4[0-1]")

In [ ]:
df.head()

In [ ]:
import ast

def getList(string):
    l = ast.literal_eval(string)
    return l

df['words'] = df['ms_sentence'].apply(getList)
df['tags'] = df['ms_disfl'].apply(getList)
print(df['tags'])

In [ ]:
df.drop(['ms_sentence', 'ms_disfl'], axis=1, inplace=True)
print(df)

In [ ]:
def changeTags(tags):
    for i in range(len(tags)):
        if tags[i] == 'C' or tags[i] == 'O':
            tags[i] = 'O'
        else:
            tags[i] = 'I'
    return tags

df['tags'] = df['tags'].apply(changeTags)
print(df['tags'])

In [ ]:
df.head()

In [ ]:
count = 0
for i in df.tags:
    if all(ele=='O' for ele in i):
        continue
    else:
        count += 1

In [ ]:
count

In [ ]:
import re

def remove_partial_and_fp(row):
    words = row['words']
    tags = row['tags']
    n_tags = []
    n_words = []
    i = 0
    j = 0
    while i < len(words):
        removed = re.sub(r'[/-]+', "", words[i])
        if words[i] == 'uh' or words[i] == 'um':
            i += 1
            j += 1
        # elif i<len(words)-1 and words[i] == 'i' and words[i+1] == 'mean':
        #     n_words.append(words[i]+ " " +words[i+1])
        #     n_tags.append(tags[j+1])
        #     i += 2
        #     j += 2
        # elif i<len(words)-1 and words[i] == 'you' and words[i+1] == 'know':
        #     n_words.append(words[i]+" "+words[i+1])
        #     n_tags.append(tags[j+1])
        #     i += 2
        #     j += 2
        elif len(removed) == 0:
            i += 1
        elif words[i][-1] == '-':
            i += 1
            j += 1
        else:
            n_words.append(words[i])
            n_tags.append(tags[j])
            i += 1
            j += 1

    row['words'] = n_words
    row['tags'] = n_tags
    return row

df = df.apply(remove_partial_and_fp, axis=1)
print(df)

In [ ]:
mask = df['words'].notnull()
df.loc[mask,'words'] = [' '.join(map(str, x)) for x in df.loc[mask,'words']]
mask = df['tags'].notnull()
df.loc[mask,'tags'] = [' '.join(map(str, x)) for x in df.loc[mask,'tags']]


In [ ]:
z=df
def checkLength(x):
    return len(x[0].split(' '))==len(x[1].split(' '))


z['length'] = z.apply(checkLength,axis=1)
z.drop(z[ z['length'] == False].index, inplace=True)
z.reset_index(drop=True)
print(z)

In [ ]:
cols = ['words','tags']
df = df[cols]

In [ ]:
test = df[rows3].reset_index(drop=True)
dev = (df[rows2]).reset_index(drop=True)
train = (df[rows1]).reset_index(drop=True)

In [ ]:
# unlabel = train.drop(['tags','none1','none2'], axis=1).reset_index(drop=True)
# unlabel['none1']='None'
# unlabel['none2']='None'
# unlabel['none3']='None'

# unlabel

In [ ]:
train.to_csv('train.txt', sep='\t', header=False, index=False)
dev.to_csv('dev.txt', sep='\t', header=False, index=False) 
test.to_csv('test.txt', sep='\t', header=False, index=False)
# unlabel.to_csv('unlabel.tsv',sep='\t', header=False)

In [ ]:
# !mv train.txt /content/drive/MyDrive/btp
# !mv test.txt /content/drive/MyDrive/btp
# !mv dev.txt /content/drive/MyDrive/btp
# !mv unlabel.tsv /content/drive/MyDrive/btp

## Use the following code to convert IO to BIO format

In [ ]:
file = open('test.txt','r',encoding='utf-8')
file = file.readlines()

In [ ]:
file2 = open('Data_IO_to_BIO/test.txt','a',encoding='utf-8')

In [ ]:
count = 0
for i,line in tqdm(enumerate(file)):
    tmp = line.split('\t')
    sent = tmp[0].split()
#     print(sent)
    tags = tmp[1].strip().split()
    for i in range(len(sent)):
        if tags[i] == "I":
            if i == 0:
                tags[i] = "B-Target"
            elif tags[i-1] == "O":
                tags[i] = "B-Target" 
            else:
                tags[i] =  tags[i]+"-Target"
        file2.write(sent[i]+"\t"+tags[i]+"\n")
    file2.write("\n")

In [ ]:
file2.close()

## To add the heads in the dataset

In [ ]:
from supar import Parser
from tqdm import tqdm

In [ ]:
parser = Parser.load('biaffine-dep-en')

In [ ]:
file = open('Data_IO_to_BIO/test.txt','r',encoding='utf-8')

In [ ]:
file = file.readlines()

In [ ]:
texts = []
labels = []
tokens = []
tags = []
for line in tqdm(file):
    if len(line.strip()) == 0:
        if len(tokens) != 0:
            texts.append(tokens)
            labels.append(tags)
            tokens = []
            tags = []
    else:
        if len(line.strip()) != 0:
            sp = line.split()
            tokens.append(sp[0])
            tags.append(sp[1])

In [ ]:
sents = []
for i in texts:
    sents.append(" ".join(i))

In [ ]:
dataset = parser.predict(sents, lang='en', prob=True, verbose=False)

In [ ]:
file = open("../Data_graph_temp/train_without_labels.txt",'a',encoding='utf-8')

In [ ]:
for i in range(len(dataset)):
    file.write(str(dataset[i]))
    file.write("\n")
file.close()

In [ ]:
file = open("../Data_graph_temp/train_without_labels.txt",'r',encoding='utf-8')
file = file.readlines()

In [ ]:
file1 = open("../Data_graph_temp/train_labels_temp.txt",'a',encoding='utf-8')

In [ ]:
tags = []
for i in labels:
    for j in i:
        tags.append(j)

In [ ]:
j = -1
k = 0
for i,line in enumerate(file):
    if len(line.strip()) == 0:
        file1.write("\n")
        k = 0
    elif line.startswith("1\t"):
        if k >= len(labels[j]):
            continue
        j += 1
        file1.write(line.strip()+"\t"+labels[j][k]+"\n")
        k += 1
    else:
        if k >= len(labels[j]):
            continue
        file1.write(line.strip()+"\t"+labels[j][k]+"\n")
        k += 1

In [ ]:
file1.close()

In [ ]:
file = open("../Data_graph_temp/train_labels_temp.txt",'r',encoding='utf-8')

In [ ]:
file = file.readlines()

In [ ]:
nums = []
words = []
lols1 = []
poss1 = []
poss2 = []
lols2 = []
headss = []
relss = []
lols3 = []
lols4 = []
labless = []

num = []
word = []
lol1 = []
pos1 = []
pos2 = []
lol2 = []
heads = []
rels = []
lol3 = []
lol4 = []
lables = []
for line in tqdm(file):
#     print(line)
    if line.strip() == "":
        nums.append(num)
        words.append(word)
        lols1.append(lol1)
        poss1.append(pos1)
        poss2.append(pos2)
        lols2.append(lol2)
        headss.append(heads)
        relss.append(rels)
        lols3.append(lol3)
        lols4.append(lol4)
        labless.append(lables)
        num = []
        word = []
        lol1 = []
        pos1 = []
        pos2 = []
        lol2 = []
        heads = []
        rels = []
        lol3 = []
        lol4 = []
        lables = []
    else:
        temp = line.strip().split("\t")
        num.append(temp[0])
        word.append(temp[1])
        lol1.append(temp[2])
        pos1.append(temp[3])
        pos2.append(temp[4])
        lol2.append(temp[5])
        heads.append(temp[6])
        rels.append(temp[7])
        lol3.append(temp[8])
        lol4.append(temp[9])
        lables.append(temp[10])

In [ ]:
file2 = open("../Data_BIO/test.txt", "a", encoding='utf-8')

In [ ]:
for i in range(len(nums)):
    for j in range(len(nums[i])):
        if int(headss[i][j]) > len(nums[i]):
            headss[i][j] = int(headss[i][j]) - 3
        if int(headss[i][j]) == len(nums[i]):
            headss[i][j] = int(headss[i][j]) - 2

In [ ]:
for i in range(len(nums)):
    for j in range(len(nums[i])):
        file2.write(nums[i][j]+"\t"+words[i][j]+"\t"+lols1[i][j]+"\t"+poss1[i][j]+"\t"+poss2[i][j]+"\t"+lols2[i][j]+"\t"+str(headss[i][j])+"\t"+relss[i][j]+"\t"+lols3[i][j]+"\t"+lols4[i][j]+"\t"+labless[i][j]+"\n")
    file2.write("\n")

In [ ]:
file2.close()

#### Data obtained now in the "Data_BIO" folder is now ready to be run through the dataprocess/bio2spannerformat.py file.